
        select bank_id, transaction_type_chain, swtc+swtv total, wight
        from(
            select bank_id, transaction_type_chain, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
            from(
                select bank_id, transaction_type_chain, sum(new_count) wight, transaction_cost, transaction_value
                from(
                    select *
                    from(
                        select bank_id, transaction_type_chain,  count(count) new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count = 0
                        group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b0
 
                    union all
                    select *
                    from(
                        select bank_id, transaction_type_chain,  count new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count != 0) as b1) as b2
                group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b3
            group by bank_id, transaction_type_chain) as b4
            where bank_id = {{number_of_bank_id}}
            order by total desc
            

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('../../github_source/tranasaction_dataset.csv', parse_dates=['time_internal'])

In [4]:
tmp_data = data[data['count'] == 0]
dg1 = tmp_data.groupby(by=['bank_id', 'transaction_type_chain', 'transaction_cost', 'transaction_value']).agg({'count': 'count'}).reset_index()

In [5]:
tmp_data2 = data[data['count'] != 0]
dg2 = tmp_data2[['count', 'bank_id', 'transaction_type_chain', 'transaction_cost', 'transaction_value']]

In [6]:
dg = pd.concat([dg1, dg2])

In [7]:
dg = dg.groupby(by=['bank_id', 'transaction_type_chain', 'transaction_cost', 'transaction_value']).agg({'count': 'sum'}).reset_index()

In [8]:
dg = dg.groupby(by=['bank_id', 'transaction_type_chain']).agg({
    'count' : 'sum',
    'transaction_cost' : 'sum',
    'transaction_value' : 'sum'
}).reset_index()

In [9]:
dg['total_value'] = dg['transaction_cost'] + dg['transaction_value']
dg = dg.drop(['transaction_cost', 'transaction_value'], axis='columns')

In [10]:
your_bank_id = 13335
dg[dg['bank_id'] == your_bank_id].sort_values('total_value', ascending=False)

bank_id                             transaction_type_chain   count  \
19551    13335             28392p;28691p;17s;28391m;27333m;27714m   16703   
20863    13335               28392p;28691p;6s;28391m;27333m;1122s  166205   
20587    13335  28392p;28691p;6s;27714m;1122s;28391m;27333m;28...    4711   
20709    13335        28392p;28691p;6s;27714m;28391m;27333m;1122s   58610   
20004    13335        28392p;28691p;28497m;6s;28391m;27333m;1122s   45953   
...        ...                                                ...     ...   
20332    13335     28392p;28691p;600p;617s;17s;28391m;27333m;882p       5   
20054    13335                     28392p;28691p;28692p;28497m;6s       5   
20257    13335  28392p;28691p;600p;617s;17s;28391m;27333m;2781...       2   
20203    13335  28392p;28691p;600p;617s;17s;28391m;27333m;2748...       5   
19497    13335  28392p;1122s;27714m;6s;28391m;27333m;27344p;27...      19   

       total_value  
19551  18537082219  
20863   8571626945  
20587   8185889315  
20709   4989932846  
20004   4025609887  
...            ...  
20332          375  
20054          370  
20257          370  
20203          365  
19497            0  

[1554 rows x 4 columns]